In [78]:
import pandas as pd

traindf = pd.read_csv('../train.csv')
testdf = pd.read_csv('../test.csv')
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
togeth = pd.concat([testdf,traindf])
homeplanets = ['Earth', 'Europa', 'Mars']

## datasplits 

In [79]:

def splits(dataframe):
    
    dataframe['Group'] = dataframe['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    dataframe['GroupNumber'] = dataframe['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    

    dataframe[['Deck', 'Number', 'Side']] = dataframe['Cabin'].str.split("/", expand=True)

    dataframe[['Group','GroupNumber']] = dataframe['PassengerId'].str.split("_", expand = True)

    togeth['Name'] = togeth['Name'].astype(str)
    togeth[['FirstName','LastName']] = togeth['Name'].str.split(" ",expand = True)


    dataframe[['Group', 'GroupNumber','Number']] = dataframe[['Group', 'GroupNumber','Number']].astype(float)

    return dataframe

togeth = splits(togeth)


## Certain imputes

#### Same name implies home planet

write how many there are beforehand with same last name and therefore home planet 

In [72]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','LastName'])

# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('LastName')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12398
Number of people from groups where not all share the same HomePlanet: 0


#### same group implies homeplanet 

In [71]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','Group'])


# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('Group')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12682
Number of people from groups where not all share the same HomePlanet: 0


#### impute findings

In [ ]:
def fill_home_planet_by_last_name(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

def fill_home_planet_by_group(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['Group'] == row['Group']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df


In [ ]:
def home_planet_from_cabindeck_abc(df):
    df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df
def home_planet_from_cabindeck_g(df):
    df.loc[df['CabinDeck']== 'G', 'HomePlanet'] = df.loc[df['CabinDeck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df


## homeplanet implies cabindeck

In [92]:
for home in homeplanets:
    print("\n" + home)
    print(togeth[togeth.HomePlanet == home].Deck.value_counts())


Earth
Deck
G    3700
F    2426
E     583
Name: count, dtype: int64

Europa
Deck
B    1124
C    1081
A     346
D     296
E     197
T      10
Name: count, dtype: int64

Mars
Deck
F    1713
E     508
D     406
Name: count, dtype: int64


People from Earth are only in decks G, F, E
People from Europa are only in decks B,C,A,D,E,T
people from Mars are only in decks F,E,D